In [ ]:
# nltk.download('stopwords')
# nltk.download('wordnet')

In [ ]:
# !python -m spacy download en_core_web_sm
# !python -m spacy download en
# !python -m spacy download en_core_web_md

In [1]:
import pandas as pd
import os
from bs4 import BeautifulSoup
import requests
import re
import nltk
import textcleaner
import pickle
import spacy
import jieba

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize, sent_tokenize
from textblob import TextBlob
import textcleaner as tc
from nltk.stem import WordNetLemmatizer 
import re
import spacy

## Load data

In [116]:
# Load scraped job description data

file_list = [
'data_110619_seattle_0to1k.pkl',
'data_110619_seattle_1kto2k.pkl',
'data_110619_seattle_2kto3k.pkl',
'data_110619_seattle_neg_5.pkl',
'data_110619_seattle_3kto4k.pkl',
'data_110619_seattle_4ktoend.pkl',
'data_110619_bellevue_0to1k.pkl',
'data_110619_bellevue_1to2k.pkl',
'data_110619_bellevue_2to3k.pkl',
'data_110619_bellevue_3toend.pkl',
'data_110619_seattle_neg.pkl'
            ]
jobs = pd.DataFrame()

for file in file_list:
    
    folder = '/Users/greenapple/project4/data/interim/'
    path = os.path.join(folder, file)
    
    pickling_out = open(path, 'rb')
    df_piece = pickle.load(pickling_out)
    jobs = pd.concat([jobs, df_piece]) 
    
    if file == 'data_110619_seattle_neg.pkl':
        jobs['type'] = 'negative'
    else:
        jobs['type'] = 'positive'
                        

/Users/greenapple/anaconda3/envs/project4e/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [117]:
jobs.shape

(13029, 7)

In [118]:
# Remove duplicates and data rows that were inserted by indeed
jobs.drop_duplicates(inplace=True)
a = jobs.shape
jobs.drop(jobs.loc[jobs.company_name == 'Seen by Indeed'].index, inplace=True)
jobs.reset_index(drop=True, inplace=True)
b = jobs.shape
print(a, b)

(13003, 7) (8937, 7)


In [17]:
# Collect data for the rows replaced by indeed
# replaced_data_0to1k = data_110619_seattle_set1.loc[data_110619_seattle_set1.company_name=='Seen by Indeed']

### Functions

In [72]:
# Pre-process text
def pre_process(text):
    '''
    Takes a string, returns a cleaned string.
    '''
    text_sub = re.sub('[/.\n:-]', ' ', text)  # Remove seaparators of dif kind
    text_sub = re.sub('[,\(\)]', '', text_sub).lower()  # Remove "," and capital letters
    text_sub = re.sub('[^a-z\s]', '', text_sub) # Keep alphanumeric characters only
#     re.sub('[%s]' % re.escape(string.punctuation), ' ', my_text)

    return text_sub

In [69]:
# Pre-process text
from gensim.parsing.preprocessing import preprocess_string, strip_numeric, strip_short, stem_text, strip_tags, strip_punctuation, strip_multiple_whitespaces, remove_stopwords

def gensim_preprocess(text):
    '''
    Takes a string, returns a list of cleaned strings.
    '''
    filters = [
        strip_tags,
        strip_punctuation, 
        strip_multiple_whitespaces, 
        strip_numeric, 
        remove_stopwords, 
        strip_short,
        lambda x: x.lower()]
    
    text_proc = preprocess_string(text, filters)
    return text_proc

In [22]:
# Tokenize text
def tokenize_lemma(text):
        
    # processer = spacy.load('en')
    tokenizer = spacy.load('en_core_web_sm')

    text_obj = tokenizer(text)
    text_str = ' '.join([token.lemma_ for token in text_obj if not token.is_stop])
    return text_str

In [203]:
# Tokenize text: nouns

tokenizer = spacy.load('en_core_web_sm')
    
def tokenize_noun(text):
        
    text_obj = tokenizer(text, disable=['parser', 'ner'])
    
    text_noun = ' '.join([token.text for token in text_obj if token.pos_=='NOUN'])
  
    return text_noun

In [205]:
# Tokenize text: lemma and nouns

tokenizer = spacy.load('en_core_web_sm')
    
def tokenize_noun_lemma(text):
        
    text_obj = tokenizer(text, disable=['parser', 'ner'])
    
    text_noun_lemma = ' '.join([token.lemma_ for token in text_obj if token.pos_=='NOUN'])
   
  
    return text_noun_lemma

In [212]:
# Tokenize text: lemma

tokenizer = spacy.load('en_core_web_sm')
    
def tokenize_lemma(text):
        
    text_obj = tokenizer(text, disable=['parser', 'ner'])
    
    text_lemma = ' '.join([token.lemma_ for token in text_obj if not token.is_stop])
  
    return text_lemma

In [213]:
# Tokenize text: words

tokenizer = spacy.load('en_core_web_sm')
    
def tokenize_word(text):
        
    text_obj = tokenizer(text, disable=['parser', 'ner'])
    
    text_text = ' '.join([token.text for token in text_obj if not token.is_stop])
  
    return text_text

### Clean text, tokenize

In [125]:
# Clean
jobs['clean_text'] = jobs['description'].apply(pre_process) # clean

# Save cleaned text
pickling_in = open('/Users/greenapple/project4/data/processed/jobs_clean.pkl', 'wb')
pickle.dump(jobs, pickling_in)

In [215]:
# Tokenize:
jobs['noun'] = jobs['clean_text'].apply(tokenize_noun)
jobs['noun_lemma'] = jobs['clean_text'].apply(tokenize_noun_lemma)
jobs['lemma'] = jobs['clean_text'].apply(tokenize_lemma)
jobs['word'] = jobs['clean_text'].apply(tokenize_word)

In [216]:
jobs.columns

Index(['company_name', 'description', 'job_title', 'link', 'location',
       'salary', 'type', 'clean_text', 'noun', 'noun_lemma', 'lemma', 'word'],
      dtype='object')

In [217]:
# Save tokenized text
pickling_in = open('/Users/greenapple/project4/data/processed/jobs_tokenized.pkl', 'wb')
pickle.dump(jobs, pickling_in)